In [24]:
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import chisquare, chi2_contingency
import matplotlib.pyplot as plt
import json

In [2]:
data = pd.read_csv("../Datasets/FinalMergedDataset/cleaned_dataset.csv")
data = pd.DataFrame(data)

In [5]:
#Printing all data in the dataset
data

,Source,Destination,Flight Name,type,Status,Time,Actual_Time,Delay
0,Paris (CDG),Mumbai,Air France,A,0,2020-02-02 00:05:00,2020-02-02 00:48:00,43.0
1,Hyderabad (HYD),Mumbai,IndiGo,A,0,2020-02-02 00:05:00,2020-02-02 00:23:00,18.0
2,Mumbai,Delhi (DEL),GoAir,D,1,2020-02-02 00:05:00,2020-02-02 00:30:00,25.0
3,Mumbai,Delhi (DEL),AirAsia,D,1,2020-02-02 00:05:00,2020-02-02 00:55:00,50.0
4,Mumbai,Bangkok (BKK),Bangkok Airways (Guilin Livery),D,1,2020-02-02 00:05:00,2020-02-02 00:50:00,45.0
...,...,...,...,...,...,...,...,...
24622,Mumbai,Ahmedabad (AMD),Emirates SkyCargo,D,1,2020-02-29 23:55:00,2020-03-01 00:23:00,28.0
24623,Mumbai,Toronto (YYZ),Air Canada,D,1,2020-02-29 23:55:00,2020-03-01 01:59:00,124.0
24624,Mumbai,Singapore (SIN),Vistara,D,1,2020-02-29 23:55:00,2020-02-29 23:56:00,1.0
24625,Mumbai,Singapore (SIN),Air India,D,1,2020-02-29 23:58:00,2020-03-01 01:09:00,71.0


In [6]:
#Printing starting 15 rows in the dataset
data.head(15)

,Source,Destination,Flight Name,type,Status,Time,Actual_Time,Delay
0,Paris (CDG),Mumbai,Air France,A,0,2020-02-02 00:05:00,2020-02-02 00:48:00,43.0
1,Hyderabad (HYD),Mumbai,IndiGo,A,0,2020-02-02 00:05:00,2020-02-02 00:23:00,18.0
2,Mumbai,Delhi (DEL),GoAir,D,1,2020-02-02 00:05:00,2020-02-02 00:30:00,25.0
3,Mumbai,Delhi (DEL),AirAsia,D,1,2020-02-02 00:05:00,2020-02-02 00:55:00,50.0
4,Mumbai,Bangkok (BKK),Bangkok Airways (Guilin Livery),D,1,2020-02-02 00:05:00,2020-02-02 00:50:00,45.0
5,Jaipur (JAI),Mumbai,GoAir,A,0,2020-02-02 00:10:00,2020-02-02 00:57:00,47.0
6,Delhi (DEL),Mumbai,IndiGo,A,0,2020-02-02 00:10:00,2020-02-01 23:45:00,25.0
7,Mumbai,New York (EWR),United Airlines,D,1,2020-02-02 00:10:00,2020-02-02 01:02:00,52.0
8,London (LHR),Mumbai,British Airways (GREAT Festival of Creativity),A,0,2020-02-02 00:15:00,2020-02-02 01:21:00,66.0
9,Dubai (DXB),Mumbai,Vistara,A,0,2020-02-02 00:15:00,2020-02-02 00:01:00,14.0


In [7]:
#Description of the dataset
data.describe()

,Status,Delay
count,24627.000000,24627.000000
mean,-345.416169,27.387948
std,1827.498152,34.347367
min,-9999.000000,0.000000
25%,0.000000,8.000000
50%,0.000000,20.000000
75%,1.000000,36.000000
max,4.000000,1178.000000


In [8]:
#Printing number of flights per flight company
data["Flight Name"].value_counts()

IndiGo                                            7381
SpiceJet                                          3917
Air India                                         3594
GoAir                                             2505
Vistara                                           2236
                                                  ... 
Bangkok Airways                                      1
Air Canada Express                                   1
Emirates (Rugby World Cup Japan 2019 Livery)         1
Air India Express (Ragini Todi-Solah Shringar)       1
Rwandair                                             1
Name: Flight Name, Length: 123, dtype: int64

In [10]:
# 5 point summary of delay
Q1, median, Q3 = np.nanpercentile(data["Delay"], [25, 50, 75])
min, max = data["Delay"].min(), data["Delay"].max()

print(
        "1. Minimum Delay = {} \n2. Maximum Delay = {} \n3. Q1 = {}\n4. Median = {}\n5. Q3 = {}".format(
            min, max, Q1, median, Q3
        )
    )

1. Minimum Delay = 0.0 
2. Maximum Delay = 1178.0 
3. Q1 = 8.0
4. Median = 20.0
5. Q3 = 36.0


In [11]:
#Mode of delay calculated
data["Delay"].mode()

0    0.0
dtype: float64

In [12]:
#Mean delay across the entire dataset
data["Delay"].mean()

27.387948186949284

In [22]:
#Standard Deviation of delay across the entire dataset
data["Delay"].std()

34.34736724414961

In [23]:
#Variance of delay
data["Delay"].var()

1179.7416366044813

In [14]:
#Finding coorelation between numerical attributes using the pearson's coefficient
cor = data.corr(method="pearson")
cor

,Status,Delay
Status,1.000000,0.150994
Delay,0.150994,1.000000


In [15]:
#Generating a frequency table of the flight status and the flight name
table = pd.crosstab(data["Flight Name"],data["Status"])
table

Status,-9999,-1,0,1,2,3,4
Flight Name,,,,,,,
Air Arabia,6,0,51,50,0,0,0
Air Canada,9,0,16,15,0,0,0
Air Canada Express,0,0,0,1,0,0,0
Air China,0,0,1,1,7,0,0
Air France,0,0,17,16,0,0,0
...,...,...,...,...,...,...,...
Virgin Atlantic,0,0,18,19,0,0,0
Vistara,108,0,1112,1016,0,0,0
Vistara (Retro Livery),0,0,28,27,0,0,0


In [16]:
# Chi Squared test to see correlation between Flight Name and Status of the flight
chi, p, df1, expected = chi2_contingency(table)
print("Chi-Squared Value = {0:3f} \n p value = {1:3f} ".format(chi, p))

Chi-Squared Value = 9507.871540 
 p value = 0.000000 


In [17]:
#Grouping data according to Flight Name
flight_data = data.groupby(["Flight Name"])
flight_data.describe()

Status                                              \
                         count         mean          std     min  25%  50%   
Flight Name                                                                  
Air Arabia               107.0  -560.224299  2311.370521 -9999.0  0.0  0.0   
Air Canada                40.0 -2249.400000  4228.802369 -9999.0  0.0  0.0   
Air Canada Express         1.0     1.000000          NaN     1.0  1.0  1.0   
Air China                  9.0     1.666667     0.707107     0.0  2.0  2.0   
Air France                33.0     0.484848     0.507519     0.0  0.0  0.0   
...                        ...          ...          ...     ...  ...  ...   
Virgin Atlantic           37.0     0.513514     0.506712     0.0  0.0  1.0   
Vistara                 2236.0  -482.502683  2144.373060 -9999.0  0.0  0.0   
Vistara (Retro Livery)    55.0     0.490909     0.504525     0.0  0.0  0.0   
Yemenia                    6.0     2.000000     0.000000     2.0  2.0  2.0   
flydubai                  31.0     0.516129     0.508001     0.0  0.0  1.0   

                                   Delay                                       \
                        75%  max   count        mean        std    min    25%   
Flight Name                                                                     
Air Arabia              1.0  1.0   107.0   24.607477  24.549433    0.0    8.0   
Air Canada              1.0  1.0    40.0   32.350000  33.944563    0.0    4.0   
Air Canada Express      1.0  1.0     1.0  155.000000        NaN  155.0  155.0   
Air China               2.0  2.0     9.0    3.111111   6.173420    0.0    0.0   
Air France              1.0  1.0    33.0   23.484848  14.035939    1.0   13.0   
...                     ...  ...     ...         ...        ...    ...    ...   
Virgin Atlantic         1.0  1.0    37.0   24.702703  18.764424    1.0   11.0   
Vistara                 1.0  1.0  2236.0   24.122093  22.940070    0.0    8.0   
Vistara (Retro Livery)  1.0  1.0    55.0   21.927273  17.523375    0.0    8.0   
Yemenia                 2.0  2.0     6.0    0.000000   0.000000    0.0    0.0   
flydubai                1.0  1.0    31.0   19.322581  20.807670    1.0    5.0   

                                             
                          50%    75%    max  
Flight Name                                  
Air Arabia               19.0   37.5  157.0  
Air Canada               22.5   48.5  124.0  
Air Canada Express      155.0  155.0  155.0  
Air China                 0.0    0.0   14.0  
Air France               21.0   32.0   63.0  
...                       ...    ...    ...  
Virgin Atlantic          23.0   32.0   65.0  
Vistara                  18.0   34.0  238.0  
Vistara (Retro Livery)   17.0   30.0   74.0  
Yemenia                   0.0    0.0    0.0  
flydubai                 18.0   24.0  101.0  

[123 rows x 16 columns]

In [18]:
#Describing delay of the grouped data
flight_data["Delay"].describe()

,count,mean,std,min,25%,50%,75%,max
Flight Name,,,,,,,,
Air Arabia,107.0,24.607477,24.549433,0.0,8.0,19.0,37.5,157.0
Air Canada,40.0,32.350000,33.944563,0.0,4.0,22.5,48.5,124.0
Air Canada Express,1.0,155.000000,NaN,155.0,155.0,155.0,155.0,155.0
Air China,9.0,3.111111,6.173420,0.0,0.0,0.0,0.0,14.0
Air France,33.0,23.484848,14.035939,1.0,13.0,21.0,32.0,63.0
...,...,...,...,...,...,...,...,...
Virgin Atlantic,37.0,24.702703,18.764424,1.0,11.0,23.0,32.0,65.0
Vistara,2236.0,24.122093,22.940070,0.0,8.0,18.0,34.0,238.0
Vistara (Retro Livery),55.0,21.927273,17.523375,0.0,8.0,17.0,30.0,74.0


In [19]:
#Finding Standard Deviation of delay of the grouped data
flight_data["Delay"].std().reset_index()

,Flight Name,Delay
0,Air Arabia,24.549433
1,Air Canada,33.944563
2,Air Canada Express,NaN
3,Air China,6.173420
4,Air France,14.035939
...,...,...
118,Virgin Atlantic,18.764424
119,Vistara,22.940070
120,Vistara (Retro Livery),17.523375
121,Yemenia,0.000000


In [20]:
#Finding Aggregated Mean of the grouped data
flight_data["Delay"].aggregate(np.mean).reset_index()

,Flight Name,Delay
0,Air Arabia,24.607477
1,Air Canada,32.350000
2,Air Canada Express,155.000000
3,Air China,3.111111
4,Air France,23.484848
...,...,...
118,Virgin Atlantic,24.702703
119,Vistara,24.122093
120,Vistara (Retro Livery),21.927273
121,Yemenia,0.000000


In [21]:
#Finding total null values in the entire dataset
data.isnull().sum()

Source            0
Destination       0
Flight Name       0
type              0
Status            0
Time              0
Actual_Time    1549
Delay             0
dtype: int64

In [24]:
###
###
###

In [7]:
#Data Visualisation

In [14]:
#Box plot of Delay for arrival and departure 
data = pd.read_csv("../Datasets/FinalMergedDataset/cleaned_dataset.csv")

delay_data = pd.DataFrame(data[["Delay","type"]])

In [17]:
arrival = delay_data[delay_data["type"] == "A"]
departure = delay_data[delay_data["type"] == "D"]

In [18]:
arrival = arrival[arrival["Delay"] <= 400 ]
departure = departure[departure["Delay"] <= 400 ]

In [25]:
import plotly.graph_objects as go
import json
import numpy as np

fig = go.Figure()
fig.add_trace(go.Box(y=arrival["Delay"]))
fig.add_trace(go.Box(y=departure["Delay"]))

fig.show()

In [20]:
location_data = data[["Source","Destination"]]
location_data

,Source,Destination
0,Paris (CDG),Mumbai
1,Hyderabad (HYD),Mumbai
2,Mumbai,Delhi (DEL)
3,Mumbai,Delhi (DEL)
4,Mumbai,Bangkok (BKK)
...,...,...
25550,Mumbai,Chennai (MAA)
25551,Colombo (CMB),Mumbai
25552,Ahmedabad (AMD),Mumbai
25553,Ahmedabad (AMD),Mumbai


In [26]:
with open("../Resources/coords.json", "r") as json_file:
    unique_location = json.load(json_file)


def source_lat(data):
    if data in unique_location.keys():
        return unique_location[data]["latitude"]
    else:
        return np.nan


def source_long(data):
    if data in unique_location.keys():
        return unique_location[data]["longitude"]
    else:
        return np.nan


def des_lat(data):
    if data in unique_location.keys():
        return unique_location[data]["latitude"]
    else:
        return np.nan


def des_long(data):
    if data in unique_location.keys():
        return unique_location[data]["longitude"]
    else:
        return np.nan


location_data["source_lat"] = location_data["Source"].apply(source_lat, 1)
location_data["source_long"] = location_data["Source"].apply(source_long, 1)
location_data["des_lat"] = location_data["Destination"].apply(des_lat, 1)
location_data["des_long"] = location_data["Destination"].apply(des_long, 1)

airport_data = location_data.groupby(
    ["Source", "Destination", "source_lat", "source_long", "des_lat", "des_long"]).size()
airport_data = airport_data.reset_index()
airport_data.columns = ("Source", "Destination", "source_lat",
                        "source_long", "des_lat", "des_long", "count")

C:\Users\2017j\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\2017j\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\2017j\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [29]:
import random

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    lon=airport_data["source_long"],
    lat=airport_data["source_lat"],
    hoverinfo="text",
    text=airport_data["Source"],
    mode="markers",
    marker=dict(
        size=10,
        color="rgb(255,0,0)",
        line=dict(
            width=3,
            color="rgba(68,68,68,0)"
        )
    )
))
fig.add_trace(go.Scattergeo(
    lon=airport_data["des_long"],
    lat=airport_data["des_lat"],
    hoverinfo="text",
    text=airport_data["Destination"],
    mode="markers",
    marker=dict(
        size=10,
        color="rgb(52,55,235)",
        line=dict(
            width=3,
            color="rgba(68,68,68,0)"
        )
    )
))
# fig.show()

for i in range(len(airport_data)):
    fig.add_trace(
        go.Scattergeo(
            lon=[airport_data["source_long"][i], airport_data["des_long"][i]],
            lat=[airport_data["source_lat"][i], airport_data["des_lat"][i]],
            mode="lines",
            line=dict(width=1, color="rgb({},{},{})".format(random.randint(
                0, 255), random.randint(0, 255), random.randint(0, 255))),
            # opacity = float(airport_data["count"][i])/float(airport_data["count"].max())
        )
    )
    

fig.show()